In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize as opt
from typing  import Tuple

In [ ]:
import akagi
import forward

In [ ]:
num_cells = 9

In [ ]:
n = int(num_cells**0.5)
x = np.linspace(-1, 1, n)
y = np.linspace(-1, 1, n)
xv, yv = np.meshgrid(x, y, sparse=False, indexing='ij')

loc = []
for i in range(n):
    for j in range(n):
        loc.append([x[i], y[j]])

loc = np.array(loc)

In [ ]:
loc

In [ ]:
rad = 0.5
N_init = np.exp(-((loc**2).sum(axis=1)**0.5 - rad)**2)
N_init = (N_init * 1000 * N_init.max()).astype(int)
N_init.reshape((n, n))

In [ ]:
s = np.exp((-4 * loc**2).sum(axis=1))
s.reshape((n, n))

In [ ]:
pi = N_init / N_init.max() / 10
pi.reshape((n, n))

In [ ]:
# Array of distances between cells
d = np.zeros((num_cells, num_cells))
for i in range(num_cells):
    for j in range(num_cells):
        d[i, j] = ((loc[i] - loc[j])**2).sum()**0.5
d

In [ ]:
beta = 1
K = 1.5

## Generate fake data

In [ ]:
simulator = forward.ForwardSimulator(pi, s, beta, d, K)

In [ ]:
num_steps = 10
np.random.seed(0)
N, M_true = simulator.simulate(N_init, num_steps)

In [ ]:
N

In [ ]:
plt.imshow(N)
plt.colorbar()

## Estimate movement from fake data

In [ ]:
a = akagi.Akagi(N, d, K)

In [ ]:
M_init = a.M
pi_init = a.pi
s_init = a.s
beta_init = a.beta

In [ ]:
%%prun
result = a.exact_inference()

Does `M` look similar?

In [ ]:
M_true[num_steps - 2]

In [ ]:
a.M[num_steps - 2].astype(int)

In [ ]:
np.abs((M_true - a.M)).sum()/M_true.sum()

Are there approximately the right number of people in the end?

In [ ]:
(a.M.sum(axis=2) - N[:-1])

In [ ]:
a.likelihood(M_init, pi_init, s_init, beta_init)

In [ ]:
a.likelihood(a.M, pi_init, s_init, beta_init)

Have `pi` and `s` converged well?

In [ ]:
pi

In [ ]:
a.pi

In [ ]:
s / s.max()

In [ ]:
a.s / a.s.max()

In [ ]:
beta

In [ ]:
a.beta

Save some vars

In [ ]:
np.save("N", N)
np.save("d", d)
np.save("K", K)
np.save("beta", beta)
np.save("s", s)
np.save("pi", pi)
np.save("M_true", M_true)